In [ ]:
import requests
import pandas as pd
from IPython.display import clear_output, display
import datetime

In [ ]:

URL_2023 = "https://5b8btxj9jd.execute-api.us-west-2.amazonaws.com/public/results?rid=129769&eid=41659&mid=1291&page={}&pageSize=1000sort=&category=&gender=&searchData="
URL_2022 = "https://5b8btxj9jd.execute-api.us-west-2.amazonaws.com/public/results?rid=114379&eid=30344&mid=1291&page={}&pageSize=1000&sort=&category=&gender=&searchData="

URL = URL_2022
YEAR=2022

In [ ]:
pg_index=0

count=0

df_final=None


columns = ['bib', 'lo3', 'pg', 'pc', 'dn', 'rtro', 'ot']
names = ['bib', 'country', 'gender', 'gender_age', 'name', 'position', 'time']
# bib: bib
# country: lo3
# gender? pg (m,f,x)
# gender_age: pc
# runner: dn
# overall pos: rtro
# official time: ot (in milliseconds)


while True:
    res = requests.get(URL.format(pg_index))
    data=res.json()
    
    if 'results' not in data or len(data['results'])==0:
        break
    count+=len(data['results'])
    
    clear_output()
    display(f"{pg_index} - {count}")

    df=pd.DataFrame(data['results'], columns=columns)

    if df_final is None:
        df_final=df
    else:
        df_final=pd.concat([df_final, df])

    pg_index+=1

df_final.columns = names

df_final.shape

In [ ]:
df_final.reset_index(inplace=True)
df_final.head(3)

In [ ]:
df_final= df_final[df_final['position'].notna()]
df_final.reset_index(inplace=True)

In [ ]:
df_final['time']=df_final['time'].apply(lambda row: row and (datetime.datetime(1970,1,1) + datetime.timedelta(milliseconds=int(row))).time().isoformat(timespec='seconds'))


In [ ]:
df_final.head(2)

In [ ]:
df_final[(df_final['gender_age'].str[0:1] != df_final['gender'].str.upper())]

In [ ]:
df_final['age']=df_final['gender_age'].str.replace(r"[a-zA-Z]*(\d.*)", '\\1',regex=True)
df_final.head(3)

In [ ]:
#res = requests.post(URL, json=qry, headers=headers)
df_final.to_csv(f"toronto_{YEAR}_full.csv")

In [ ]:
df_final[['country','gender','age','time']].to_csv(f"toronto_{YEAR}.csv")

In [ ]:
df_final[['gender','age']].groupby('gender', dropna=False).count()

In [ ]:
df_final[df_final.sex.isna()]